In [1]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

import pickle


In [2]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
data = pd.read_csv("spam.csv", encoding="latin-1")
data = data[['v1', 'v2']]
data.columns = ['label', 'message']
data.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
ps = PorterStemmer()
corpus = []

for msg in data['message']:
    msg = re.sub('[^a-zA-Z]', ' ', msg)
    msg = msg.lower()
    msg = msg.split()
    msg = [ps.stem(word) for word in msg if word not in stopwords.words('english')]
    msg = ' '.join(msg)
    corpus.append(msg)


In [5]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(corpus).toarray()

y = data['label'].map({'ham': 0, 'spam': 1})


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [7]:
model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

In [8]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.968609865470852


In [9]:
pickle.dump(model, open("spam_model.pkl", "wb"))
pickle.dump(tfidf, open("tfidf.pkl", "wb"))


In [13]:
%%writefile app.py
import streamlit as st
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')

model = pickle.load(open("spam_model.pkl", "rb"))
tfidf = pickle.load(open("tfidf.pkl", "rb"))
ps = PorterStemmer()

st.title("📧 Spam Email Classification")
st.write("Enter an email or message to check if it is Spam or Not")

input_msg = st.text_area("Email / Message Text")

def preprocess(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if word not in stopwords.words('english')]
    return ' '.join(text)

if st.button("Check"):
    processed_msg = preprocess(input_msg)
    vectorized_msg = tfidf.transform([processed_msg])
    prediction = model.predict(vectorized_msg)

    if prediction[0] == 1:
        st.error("🚨 This message is SPAM")
    else:
        st.success("✅ This message is NOT SPAM")

Writing app.py


In [18]:
!streamlit run app.py & npx degit github:streamlit/colab-cli/examples/run-app

⠙

⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦Need to install the following packages:
degit@2.8.4
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.234.22.113:8501



  Stopping...
^C


In [15]:
!pip install pyngrok

In [14]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.7 MB/s eta 0:00:00


In [21]:
!streamlit run app.py &>/content/logs.txt &


In [17]:
from pyngrok import ngrok

# Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual ngrok authtoken
# You can get this from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("39Bn6wd6vQBwQRYqJ4fUIOmCdPc_2opsgoKThgQWp999MRzTn")

public_url = ngrok.connect(8501)
print(public_url)

NgrokTunnel: "https://aboveground-samson-tongue.ngrok-free.dev" -> "http://localhost:8501"


In [22]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("OPEN THIS URL:", public_url)


OPEN THIS URL: NgrokTunnel: "https://aboveground-samson-tongue.ngrok-free.dev" -> "http://localhost:8501"
